Progetto di Linguistica Computazionale di Giacomo Fidone (531668) – Estrazione di informazioni da constitution_of_the_united_states.txt

# Imports

In [1]:
import nltk
from nltk import ngrams
import math

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Estrazione di Informazioni da un testo

In [3]:
class Analyze:

    def __init__(self, path):
        '''Definisce una classe per l'estrazione di informazioni da un testo.'''

        self.text = self.read_file(path)
        self.sents = nltk.tokenize.sent_tokenize(self.text)
        self.tokenized_sents = [nltk.word_tokenize(sent) for sent in self.sents]
        self.tokens = [token for sent in self.tokenized_sents for token in sent]
        self.freq_dist = nltk.FreqDist(self.tokens)
        self.pos = nltk.tag.pos_tag(self.tokens, tagset='universal') # uso il tagset di UD
        self.only_pos = [pos for token, pos in self.pos]

    def top_k_words(self, k, pos_tags=None):
        '''Mostra i k token più frequenti (e relativa frequenza) in ordine decrescente. E' possibile indicare i pos-tags da
        considerare assegnando a "pos_tags" la lista di UD pos tags. '''

        if pos_tags==None:
            freqs = self.freq_dist
        else:
            tokens = [token for token, pos in self.pos if pos in pos_tags]
            freqs = nltk.FreqDist(tokens)

        top_k = {token : freqs[token] for token in list(freqs)[:k]}
        for i, token in enumerate(top_k):
            print(f'{i+1}. {token} : {top_k[token]}')

    def top_k_ngrams(self, k, n, pos=False):
        '''Mostra i primi k n-grammi di parole più frequenti (e relative frequenze) in ordine decrescente. Se "pos"=True,
        considera n-grammi di pos.'''

        if pos:
            n_grams = list(ngrams(self.only_pos, n))
        else:
            n_grams = list(ngrams(self.tokens, n))
        voc = list(set(n_grams))
        freqs = nltk.FreqDist(n_grams)

        top_k = {n_gram : freqs[n_gram] for n_gram in list(freqs)[:k]}
        for i, n_gram in enumerate(top_k):
            print(f'{i+1}. {n_gram} : {top_k[n_gram]}')

    def top_k_bigrams(self, k, pos_tags, order='frequency', display_output=True):
        '''Mostra i primi k bigrammi di parole, le cui pos corrispondono a "pos_tags", in base a "order". Valori possibili di "order" sono: "frequency" (ordinati per frequenza – default);
        "cond_prob" (ordinati per probabilità condizionata), "joint_prob" (ordinati per probabilità congiunta), "mi" (ordinati per mutual information),
        "lmi" (ordinati per local mutual information). Il metodo è usato internamente da self.common_mi_lmi(), che disattiva il flag "display_output".'''

        bigrams = list(ngrams(self.tokens, 2))
        pos_bigrams = list(ngrams(self.only_pos, 2))
        bigrams = [bigram for bigram, pos_bigram in zip(bigrams, pos_bigrams) if pos_bigram==pos_tags] # seleziono i bigrammi le cui pos corrispodono a "pos_tags"
        bigrams_freq_dist = nltk.FreqDist(bigrams)

        results = dict() # dizionario della forma {bigramma:valore}, dove valore è il valore della misura selezionata per "order"
        if order == 'frequency':
            results = bigrams_freq_dist
        else:
            voc = list(set(bigrams)) # vocabolario di bigrammi
            for bigram in voc:
                bigram_freq = bigrams_freq_dist[bigram]
                first_token_freq = self.freq_dist[bigram[0]]
                corpus_size = len(self.tokens)
                if order == 'cond_prob' or order=='joint_prob':
                    cond_prob = bigram_freq / first_token_freq # P(B|A) = F(A, B) / F(A)
                    if order == 'cond_prob':
                        results[bigram] = cond_prob
                    else:
                        results[bigram] = (first_token_freq/corpus_size) * cond_prob # P(A, B) = P(A)P(B|A)
                else:
                    first_token_prob = first_token_freq / corpus_size
                    second_token_prob = self.freq_dist[bigram[1]] / corpus_size
                    bigram_prob = bigram_freq / len(bigrams)
                    mi = math.log(bigram_prob / (first_token_prob * second_token_prob), 2) # MI = log_2(P(A, B) / P(A) * P(B))
                    if order == 'mi':
                        results[bigram] = mi
                    elif order == 'lmi':
                        results[bigram] = bigram_freq * mi # LMI = F(A, B) * MI
                    else:
                        raise ValueError(f'Invalid value for "order". Expected one of ["frequency", "cond_prob", "joint_prob", "mi", "lmi"], got "{order}".')
            results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True)) # ordino i bigrammi in senso decrescente secondo il valore assegnato

        top_k = {bigram : results[bigram] for bigram in list(results)[:k]} # considero solo i primi k bigrammi
        if display_output:
            for i, bigram in enumerate(top_k):
                print(f'{i+1}. {bigram} : {top_k[bigram]}')
        else:
            return top_k.keys()

    def common_mi_lmi(self, k, pos_tags, display_output=False):
        '''Mostra i bigrammi in comune ai primi k bigrammi (le cui pos corrispondono a 'pos_tags') ordinati per mutual information e local mutual information.'''

        mi_bigrams = self.top_k_bigrams(k, pos_tags, order='mi', display_output=False)
        lmi_bigrams = self.top_k_bigrams(k, pos_tags, order='lmi', display_output=False)
        common = [bigram for bigram in mi_bigrams if bigram in lmi_bigrams]
        if common:
            print(f'Bigrammi in comune nei primi {k} bigrammi con pos {pos_tags} ordinati per mutual information e local mutual information:')
            for bigram in common:
                print(bigram)
        else:
            print(f'Non ci sono bigrammi in comune nei primi {k} bigrammi con pos {pos_tags} ordinati per mutual information e local mutual information.')

    def analyze_sentences(self):
        '''Mostra la frase con la media della distribuzione di frequenza massima, la frase con la media di distribuzione di frequenza minima
        e la frase con la più alta probabilità secondo un modello di markov di ordine 2. Considera solo frasi la cui lunghezza è compresa tra 10 e 20 token
        e di cui almeno metà dei token non è un hapax.'''

        target_sents = [sent for sent in self.tokenized_sents if len(sent) <=20 and len(sent) >= 10] # considero solo frasi la cui lunghezza è compresa tra 10 e 20
        for sent in target_sents:
            no_hapax_words = len([token for token in sent if self.freq_dist[token] > 1]) # calcolo il numero di parole della frase con frequenza maggiore di 1
            if no_hapax_words < (len(sent) // 2): # rimuovo la frase se il numero di parole che non sono hapax è inferiore alla metà del numero di parole totale
                target_sents.remove(sent)

        max_avg_freq = 0
        max_avg_freq_sent = str()
        min_avg_freq = float('inf')
        min_avg_freq_sent = str()
        max_markov_prob = 0
        max_markov_prob_sent = str()

        for sent in target_sents:
            token_freqs = [self.freq_dist[token] for token in sent]
            avg_freq = sum(token_freqs) / len(sent) # media della distribuzione di frequenza
            markov_prob = self.markov2(sent) # probabilità secondo modello di markov di ordine 2
            if avg_freq > max_avg_freq:
                max_avg_freq = avg_freq
                max_avg_freq_sent = sent
            if avg_freq < min_avg_freq:
                min_avg_freq = avg_freq
                min_avg_freq_sent = sent
            if markov_prob > max_markov_prob:
                max_markov_prob = markov_prob
                max_markov_prob_sent = sent

        max_avg_freq_sent = self.sents[self.tokenized_sents.index(max_avg_freq_sent)] # recupero le frasi non tokenizzate per la stampa a video
        min_avg_freq_sent = self.sents[self.tokenized_sents.index(min_avg_freq_sent)]
        max_markov_prob_sent = self.sents[self.tokenized_sents.index(max_markov_prob_sent)]

        print(f'Frase con la media della distribuzione di frequenza dei token più alta ({round(max_avg_freq, 2)}): "{max_avg_freq_sent}".', )
        print(f'Frase con la media della distribuzione di frequenza dei token più bassa ({round(min_avg_freq, 2)}): "{min_avg_freq_sent}".', )
        print(f'Frase con la probabilità più alta secondo un modello di Markov di ordine 2 ({max_markov_prob}): "{max_markov_prob_sent}".', )


    def markov2(self, tokenized_sent):
        '''Restituisce la probabilità di tokenized_sent secondo un modello di markov di ordine 2
        (con add-1 smoothing) costruito a partire dal corpus di input.'''

        len_voc = len(list(set(self.tokens)))
        prob = (self.freq_dist[tokenized_sent[0]] + 1)/ (len(self.tokens) + len_voc) # P(A_1)
        prob *= self.cond_prob((tokenized_sent[0], tokenized_sent[1])) # P(A_2|A_1)

        sent_trigrams = list(ngrams(tokenized_sent, 3))
        for trigram in sent_trigrams:
            prob *= self.cond_prob(trigram) # P(A_n|A_{n-1}, A_{n-2})
        return prob

    def cond_prob(self, n_gram):
        '''Dato un n-gramma (A_1, ..., A_n), restituisce la probabilità condizionata
        P(A_n | A_1, ..., A_{n - 1}) calcolata con add-1 smoothing.'''

        n = len(n_gram)
        n_grams = list(ngrams(self.tokens, n))
        prefix = tuple(n_gram[:-1]) # contesto
        n_minus_one_grams = list(ngrams(self.tokens, n - 1))
        prefix_freq = n_minus_one_grams.count(prefix) # frequenza del contesto
        len_voc = len(list(set(self.tokens)))
        return (n_grams.count(n_gram) + 1) / (prefix_freq + len_voc) # (F(A_1, .., A_n) + 1) / (F(A_1, ..., A_{n-1}) + |V|)

    def ner(self, k):
        '''Estrae le NE dal testo e mostra, per ciascuna classe, i k token più frequenti
        (con relativa frequenza) ordinati per frequenza decrescente.'''

        ne_tree = nltk.ne_chunk(nltk.tag.pos_tag(self.tokens)) # per NER uso le POS di Penn Treebank
        ne = dict() # dizionario della forma {entity_type:[entities]}
        for node in ne_tree: # accedo a ciascun nodo dell'albero
            if hasattr(node, 'label'): # se il nodo corrisponde ad una NE, accedo alla classe di NE e alla NE
                    entity_type = node.label()
                    entity = " ".join([token for token, pos in node.leaves()])
                    if entity_type in ne.keys():
                        ne[entity_type].append(entity)
                    else:
                        ne[entity_type] = [entity]

        for entity_type, entities in ne.items():
            print(entity_type)
            freq_dist = nltk.FreqDist(entities)
            top_k = {entity : freq_dist[entity] for entity in list(freq_dist)[:k]} # considero solo i primi k elementi
            for i, entity in enumerate(top_k):
                print(f'{i+1}. {entity} : {top_k[entity]}')
            print('')

    @staticmethod
    def read_file(path_to_file):
        '''Apre un file e lo restituisce come stringa.'''

        with open (path_to_file, 'r', encoding='utf-8') as infile:
            return infile.read()

In [4]:
analyze = Analyze('constitution_of_the_united_states.txt')

In [5]:
analyze.top_k_words(50, pos_tags=['NOUN']) # top 50 sostantivi più frequenti

1. States : 221
2. February : 162
3. January : 160
4. March : 141
5. United : 120
6. President : 118
7. Congress : 116
8. State : 103
9. amendment : 103
10. * : 91
11. New : 80
12. April : 69
13. Constitution : 68
14. June : 62
15. Section : 59
16. May : 54
17. House : 49
18. December : 49
19. Senate : 44
20. July : 44
21. Article : 43
22. Representatives : 39
23. Carolina : 39
24. legislatures : 39
25. Ratification : 38
26. Virginia : 38
27. ratification : 34
28. article : 31
29. North : 30
30. Vice : 29
31. South : 28
32. York : 24
33. [ : 24
34. September : 23
35. Massachusetts : 23
36. Hampshire : 23
37. Law : 23
38. August : 23
39. Jersey : 22
40. Connecticut : 22
41. Maryland : 22
42. time : 22
43. Delaware : 21
44. Pennsylvania : 21
45. November : 21
46. Rhode : 20
47. Island : 20
48. office : 20
49. Vermont : 19
50. Office : 19


In [6]:
analyze.top_k_words(50, pos_tags=['ADV']) # top 50 avverbi più frequenti

1. not : 67
2. subsequently : 29
3. when : 25
4. then : 13
5. so : 7
6. previously : 7
7. otherwise : 5
8. only : 4
9. respectively : 4
10. hereby : 4
11. as : 3
12. When : 3
13. once : 3
14. immediately : 3
15. whenever : 3
16. Whenever : 3
17. how : 2
18. effectually : 2
19. most : 2
20. equally : 2
21. also : 2
22. together : 2
23. prior : 2
24. before : 2
25. accordingly : 2
26. faithfully : 2
27. where : 2
28. well : 2
29. twice : 2
30. more : 1
31. ordain : 1
32. far : 1
33. essentially : 1
34. already : 1
35. promptly : 1
36. unanimously : 1
37. \2\Immediately : 1
38. nevertheless : 1
39. likewise : 1
40. square : 1
41. now : 1
42. absolutely : 1
43. enter : 1
44. actually : 1
45. neither : 1
46. solemnly : 1
47. alone : 1
48. thereof : 1
49. thereby : 1
50. ever : 1


In [7]:
analyze.top_k_words(50, pos_tags=['ADJ']) # top 50 aggettivi più frequenti

1. such : 56
2. several : 42
3. other : 37
4. * : 26
5. same : 15
6. following : 14
7. first : 14
8. valid : 14
9. necessary : 12
10. public : 12
11. whole : 10
12. more : 10
13. appropriate : 9
14. second : 8
15. respective : 7
16. foreign : 7
17. ] : 7
18. \1\The : 6
19. equal : 6
20. common : 5
21. least : 5
22. ten : 5
23. supreme : 5
24. judicial : 5
25. inoperative : 5
26. original : 4
27. uniform : 4
28. present : 4
29. new : 4
30. subject : 4
31. different : 4
32. greatest : 4
33. unable : 4
34. domestic : 3
35. general : 3
36. free : 3
37. actual : 3
38. subsequent : 3
39. fourth : 3
40. next : 3
41. civil : 3
42. executive : 3
43. highest : 3
44. eligible : 3
45. principal : 3
46. inferior : 3
47. male : 3
48. twenty-one : 3
49. three- : 3
50. less : 3


In [8]:
analyze.top_k_ngrams(20, 1) # top 20 unigrammi di parole più frequenti

1. (',',) : 2311
2. ('--',) : 1217
3. ('the',) : 1092
4. ('of',) : 816
5. (';',) : 734
6. ('.',) : 440
7. ('and',) : 372
8. ('shall',) : 333
9. ('to',) : 314
10. ('by',) : 262
11. ('States',) : 221
12. ('be',) : 210
13. ('in',) : 181
14. ('on',) : 173
15. ('or',) : 173
16. ('February',) : 162
17. ('January',) : 160
18. ('March',) : 141
19. ('a',) : 131
20. ('*',) : 130


In [9]:
analyze.top_k_ngrams(20, 2) # top 20 bigrammi di parole più frequenti

1. ('--', '--') : 1164
2. ('of', 'the') : 341
3. (',', 'January') : 139
4. (',', 'and') : 135
5. (',', 'February') : 133
6. ('the', 'United') : 119
7. ('United', 'States') : 119
8. ('shall', 'be') : 119
9. ('by', 'the') : 119
10. (',', 'March') : 116
11. ('to', 'the') : 115
12. ('*', '*') : 104
13. ('.', 'The') : 86
14. (',', '1933') : 74
15. (',', 'or') : 71
16. ('States', ',') : 69
17. (';', 'New') : 63
18. (',', 'April') : 63
19. ('1933', ';') : 63
20. (',', '1919') : 57


In [10]:
analyze.top_k_ngrams(20, 3) # top 20 trigrammi di parole più frequenti

1. ('--', '--', '--') : 1127
2. ('the', 'United', 'States') : 118
3. ('of', 'the', 'United') : 89
4. ('*', '*', '*') : 78
5. (',', '1933', ';') : 63
6. (',', '1919', ';') : 49
7. ('United', 'States', ',') : 41
8. (',', '1971', ';') : 41
9. ('the', 'several', 'States') : 36
10. ('the', 'legislatures', 'of') : 35
11. (',', '1913', ';') : 34
12. (',', '1961', ';') : 34
13. (',', '1963', ';') : 34
14. ('--', '--', '-') : 32
15. ('to', 'the', 'Constitution') : 31
16. ('ratified', 'by', 'the') : 28
17. ('of', 'the', 'several') : 28
18. (',', '1865', ';') : 28
19. ('(', 'after', 'having') : 28
20. ('after', 'having', 'rejected') : 28


In [11]:
analyze.top_k_ngrams(20, 4) # top 20 quadrigrammi di parole più frequenti

1. ('--', '--', '--', '--') : 1090
2. ('of', 'the', 'United', 'States') : 88
3. ('*', '*', '*', '*') : 52
4. ('the', 'United', 'States', ',') : 41
5. ('--', '--', '--', '-') : 32
6. ('(', 'after', 'having', 'rejected') : 28
7. ('.', '--', '--', '--') : 27
8. ('.', 'The', 'amendment', 'was') : 27
9. ('amendment', 'to', 'the', 'Constitution') : 26
10. ('after', 'having', 'rejected', 'it') : 26
11. ('of', 'the', 'several', 'States') : 25
12. ('having', 'rejected', 'it', 'on') : 25
13. ('by', 'the', 'legislatures', 'of') : 22
14. ('the', 'Constitution', 'of', 'the') : 21
15. ('Constitution', 'of', 'the', 'United') : 20
16. ('was', 'subsequently', 'ratified', 'by') : 19
17. (';', 'New', 'Hampshire', ',') : 19
18. ('Ratification', 'was', 'completed', 'on') : 19
19. ('ratified', 'by', 'the', 'legislatures') : 18
20. ('have', 'been', 'ratified', 'by') : 18


In [12]:
analyze.top_k_ngrams(20, 5) # top 20 quinquigrammi di parole più frequenti

1. ('--', '--', '--', '--', '--') : 1057
2. ('of', 'the', 'United', 'States', ',') : 32
3. ('--', '--', '--', '--', '-') : 32
4. ('.', '--', '--', '--', '--') : 27
5. ('*', '*', '*', '*', '*') : 26
6. ('(', 'after', 'having', 'rejected', 'it') : 26
7. ('after', 'having', 'rejected', 'it', 'on') : 25
8. ('the', 'Constitution', 'of', 'the', 'United') : 20
9. ('Constitution', 'of', 'the', 'United', 'States') : 20
10. ('ratified', 'by', 'the', 'legislatures', 'of') : 18
11. ('to', 'the', 'Constitution', 'of', 'the') : 17
12. ('to', 'have', 'been', 'ratified', 'by') : 17
13. ('.', 'Ratification', 'was', 'completed', 'on') : 17
14. ('.', 'The', 'amendment', 'was', 'subsequently') : 17
15. ('The', 'amendment', 'was', 'subsequently', 'ratified') : 17
16. ('amendment', 'was', 'subsequently', 'ratified', 'by') : 17
17. ('--', '--', '--', '-', '*') : 16
18. ('--', '--', '-', '*', '*') : 16
19. ('--', '-', '*', '*', '*') : 16
20. ('-', '*', '*', '*', '*') : 16


In [13]:
analyze.top_k_ngrams(20, 1, pos=True) # top 20 unigrammi di pos più frequenti

1. ('NOUN',) : 5489
2. ('.',) : 4933
3. ('VERB',) : 2096
4. ('ADP',) : 2048
5. ('NUM',) : 1885
6. ('DET',) : 1768
7. ('CONJ',) : 619
8. ('ADJ',) : 612
9. ('PRT',) : 321
10. ('PRON',) : 258
11. ('ADV',) : 245
12. ('X',) : 9


In [14]:
analyze.top_k_ngrams(20, 2, pos=True) # top 20 bigrammi di pos più frequenti

1. ('NUM', '.') : 1703
2. ('.', 'NOUN') : 1633
3. ('NOUN', '.') : 1630
4. ('DET', 'NOUN') : 1336
5. ('.', '.') : 1307
6. ('NOUN', 'ADP') : 1120
7. ('ADP', 'DET') : 972
8. ('NOUN', 'NUM') : 885
9. ('.', 'NUM') : 852
10. ('NOUN', 'NOUN') : 798
11. ('VERB', 'VERB') : 723
12. ('ADP', 'NOUN') : 676
13. ('NOUN', 'VERB') : 540
14. ('ADJ', 'NOUN') : 463
15. ('VERB', 'ADP') : 399
16. ('.', 'CONJ') : 324
17. ('.', 'ADP') : 259
18. ('DET', 'ADJ') : 240
19. ('VERB', 'DET') : 234
20. ('NOUN', 'CONJ') : 231


In [15]:
analyze.top_k_ngrams(20, 3, pos=True) # top 20 trigrammi di pos più frequenti

1. ('.', '.', '.') : 1199
2. ('NOUN', 'NUM', '.') : 865
3. ('NOUN', '.', 'NOUN') : 851
4. ('.', 'NUM', '.') : 832
5. ('NUM', '.', 'NUM') : 796
6. ('ADP', 'DET', 'NOUN') : 732
7. ('.', 'NOUN', 'NUM') : 725
8. ('NUM', '.', 'NOUN') : 664
9. ('.', 'NOUN', '.') : 569
10. ('NOUN', 'ADP', 'DET') : 562
11. ('DET', 'NOUN', 'ADP') : 519
12. ('NOUN', 'ADP', 'NOUN') : 398
13. ('NOUN', 'NOUN', '.') : 358
14. ('NOUN', 'VERB', 'VERB') : 291
15. ('DET', 'NOUN', 'VERB') : 227
16. ('NOUN', '.', 'CONJ') : 225
17. ('.', 'NOUN', 'NOUN') : 225
18. ('DET', 'NOUN', 'NOUN') : 223
19. ('VERB', 'VERB', 'ADP') : 216
20. ('DET', 'ADJ', 'NOUN') : 209


In [16]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN')) # top 10 bigrammi (composti da aggettivo e sostantivo) per frequenza

1. ('several', 'States') : 36
2. ('*', '*') : 26
3. ('appropriate', 'legislation') : 9
4. ('whole', 'number') : 7
5. (']', 'Section') : 7
6. ('following', 'States') : 5
7. ('following', 'article') : 5
8. ('supreme', 'Court') : 4
9. ('public', 'Ministers') : 4
10. ('other', 'States') : 3


In [17]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN'), order='cond_prob') # top 10 bigrammi (composti da aggettivo e sostantivo) per probabiità condizionata

1. ('seventeenth', 'amendment') : 1.0
2. ('previous', 'condition') : 1.0
3. ('legislative', 'Powers') : 1.0
4. ('sixteenth', 'amendment') : 1.0
5. ('fourteenth', 'amendment') : 1.0
6. ('appropriate', 'legislation') : 1.0
7. ('commercial', 'regulations') : 1.0
8. ('foregoing', 'Powers') : 1.0
9. ('perfect', 'Union') : 1.0
10. ('superior', 'figures') : 1.0


In [18]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN'), order='joint_prob') # top 10 bigrammi (composti da aggettivo e sostantivo) per probabiità congiunta

1. ('several', 'States') : 0.0017748853719863926
2. ('*', '*') : 0.001281861657545728
3. ('appropriate', 'legislation') : 0.00044372134299659814
4. (']', 'Section') : 0.0003451166001084653
5. ('whole', 'number') : 0.0003451166001084652
6. ('following', 'article') : 0.0002465118572203323
7. ('following', 'States') : 0.0002465118572203323
8. ('public', 'Ministers') : 0.00019720948577626584
9. ('supreme', 'Court') : 0.00019720948577626584
10. ('same', 'State') : 0.00014790711433219938


In [19]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN'), order='mi') # top 10 bigrammi (composti da aggettivo e sostantivo) per mutua informazione

1. ('previous', 'condition') : 19.76109848180614
2. ('commercial', 'regulations') : 19.76109848180614
3. ('superior', 'figures') : 19.76109848180614
4. ('unusual', 'punishments') : 19.76109848180614
5. ('net', 'Produce') : 19.76109848180614
6. ('forty-eight', 'hours') : 19.76109848180614
7. ('suppress', 'Insurrections') : 19.76109848180614
8. ('unanimous', 'conviction') : 19.76109848180614
9. ('good', 'Behaviour') : 19.76109848180614
10. ('technical', 'correction') : 19.76109848180614


In [20]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN'), order='lmi') # top 10 bigrammi (composti da aggettivo e sostantivo) per mutua informazione locale

1. ('several', 'States') : 423.0289260388173
2. ('*', '*') : 270.8368669211484
3. ('appropriate', 'legislation') : 147.95253348226902
4. ('whole', 'number') : 105.53620215273848
5. (']', 'Section') : 84.29367915309024
6. ('following', 'article') : 65.20683712528144
7. ('public', 'Ministers') : 64.70454392433993
8. ('supreme', 'Court') : 63.81497423899415
9. ('three-', 'fourths') : 51.52840794325495
10. ('following', 'States') : 51.03830588025865


In [21]:
analyze.common_mi_lmi(10, ('ADJ', 'NOUN'))

Non ci sono bigrammi in comune nei primi 10 bigrammi con pos ('ADJ', 'NOUN') ordinati per mutual information e local mutual information.


In [22]:
analyze.analyze_sentences()

Frase con la media della distribuzione di frequenza dei token più alta (495.05): "The Congress shall have the power to enforce, by 
appropriate legislation, the provisions of this article.".
Frase con la media della distribuzione di frequenza dei token più bassa (120.94): "The dates 
set out in this document are based upon the best information 
available.]".
Frase con la probabilità più alta secondo un modello di Markov di ordine 2 (2.2559210720711727e-28): "Congress shall have power to enforce this 
article by appropriate legislation.".


In [23]:
analyze.ner(15) # top 15 NE più frequenti per ciascuna classe di NE con relativa frequenza

ORGANIZATION
1. Congress : 107
2. House : 47
3. Senate : 41
4. Representatives : 33
5. States : 16
6. General Services : 10
7. Legislature : 9
8. Constitution : 8
9. Consent : 7
10. District : 7
11. THE : 6
12. Senators : 6
13. Citizens : 6
14. Validity Publication : 6
15. Houses : 5

GPE
1. United States : 119
2. New York : 23
3. Virginia : 22
4. Massachusetts : 21
5. New Hampshire : 21
6. Pennsylvania : 20
7. Maryland : 20
8. New Jersey : 18
9. Vermont : 18
10. South Carolina : 16
11. Ohio : 16
12. Georgia : 15
13. North Carolina : 15
14. Delaware : 15
15. Missouri : 15

PERSON
1. Article : 25
2. Rhode Island : 20
3. Michigan : 15
4. Minnesota : 15
5. Louisiana : 12
6. Montana : 12
7. Law : 11
8. Person : 8
9. Power : 6
10. Treason : 5
11. Bill : 5
12. Money : 4
13. Illinois : 4
14. Delaware : 3
15. States : 3

GSP
1. Connecticut : 21
2. Consequence : 3
3. Philadelphia : 1

LOCATION
1. West Virginia : 14
2. North Carolina : 1

